## Retail web scrapping

In [77]:
import os
import sys
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
# Add the parent directory to sys.path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [3]:
from outils import load_config, get_driver, load_keys

In [4]:
keys_f = load_keys('logging_key.yaml')

In [100]:
def logging(url):
    """
    This module implements the logging and scrapping process of a reatual page 
    Args:
    url (string): url to login
    """
    tenis=[]
    tenis_pd=pd.DataFrame()
    driver=get_driver(url)
    time.sleep(3)
    driver.find_element(by = "xpath",value = "/html/body/main/article/section/div/div[1]/form/div[1]/input").send_keys(keys_f["logging_mail"])
    time.sleep(3)
    driver.find_element(by = "xpath",value = "/html/body/main/article/section/div/div[1]/form/div[2]/input").send_keys(keys_f["logging_key"]+ Keys.RETURN)
    time.sleep(4)
    driver.find_element(by = "xpath", value = "/html/body/header/div[1]/div[1]/div/div[2]/nav[1]/ul/li[2]/a" ).click()
    time.sleep(4)
    try:
        for numero in range(1,100):
            numero = "/html/body/main/article/div/div/div/div[1]/section/div[3]/div[{}]/div/a".format(numero)
            element= driver.find_element(by = "xpath", value = numero).text.split("\n")
            tenis.append(element)    
    except:
        for pagina in range(0,20):
            try: 
                driver.find_element(By.LINK_TEXT, 'Next »').click()
                time.sleep(3)
                for numero in range(1,16):
                    numero = "/html/body/main/article/div/div/div/div[1]/section/div[3]/div[{}]/div/a".format(numero)
                    element= driver.find_element(by = "xpath", value = numero).text.split("\n")
                    tenis.append(element)   
            except:
                print(f"web scrapping stopped at {len(tenis)} element")
    return tenis, tenis_pd


In [101]:
path_retailer='https://titan22.com/account/login?return_url=%2Faccount'

In [102]:
tennis, df=logging(path_retailer)

web scrapping stopped at 294 element
web scrapping stopped at 294 element


In [151]:
df_tennis = pd.DataFrame(tennis, columns=['Marca', 'Modelo', 'Precio original','Precio descuento'])

In [152]:
df_tennis = (
    df_tennis
    # Extraer el numero de episodio
    .assign(
        nombre_producto = lambda df_: df_['Modelo'].str.split("'").str[0].str.strip(),
        estilo = lambda df_: df_['Modelo'].str.split("'").str[1].str.strip(),
        precio_original = lambda df_: (df_['Precio original'].str.replace('₱', '', regex=True)
                                       .str.replace('.00', '', regex=False)
                                       .str.replace(',', '', regex=False)),
        precio_descuento = lambda df_: (df_['Precio descuento'].str.replace('₱', '', regex=True)
                                       .str.replace('.00', '', regex=False)
                                       .str.replace(',', '', regex=False))
    )
    .drop(columns=['Modelo','Precio original','Precio descuento'])
)

In [153]:
df_tennis

,Marca,nombre_producto,estilo,precio_original,precio_descuento
0,ADIDAS,adidas BYW Select,Jalen Green Philippines ID,8500,None
1,JORDAN,Air Jordan 1 Retro High OG,Light Smoke Grey,9895,None
2,JORDAN,Air Jordan 1 Retro High OG GS,Light Smoke Grey,6445,None
3,PUMA,Puma MB.02,Be You,9900,None
4,NIKE,Nike LeBron XX EP,Laser Orange,10895,None
...,...,...,...,...,...
289,JORDAN,Air Jordan 2 Retro x Maison Château Rouge,Orange and Sail,11495,None
290,NIKE SPORTSWEAR,Nike Blazer Low,77 Jumbo,5095,None
291,NIKE SPORTSWEAR,Nike Air Force 1,07,5495,None
292,NIKE SPORTSWEAR,Wmns Nike Air Force 1,07 Next Nature,5495,None
